# DESS Model - Kaggle P100 GPU Setup
## Aspect Sentiment Triplet Extraction with Enhanced Semantic GCN

This notebook runs the DESS (D2E2S) model with **Enhanced Semantic GCN** for improved performance.

**Best Configuration**: Enhanced Semantic GCN only (77.14% Triplet F1)

**Note**: Contrastive learning was tested but decreased performance (-1.04%), so we use Enhanced SemGCN only.

## 1. Clone Repository

In [ ]:
!git clone https://github.com/VishalRepos/DESS-improved.git
%cd DESS-improved/Codebase
!echo "\n=== Latest Commits ==="
!git log --oneline -5

## 2. Check GPU Availability

In [ ]:
!nvidia-smi

## 3. Install Dependencies

In [ ]:
# Install compatible versions
!pip install -q numpy==1.26.4
!pip install -q transformers==4.36.0
!pip install -q torch==2.1.0
!pip install -q Jinja2==3.1.2
!pip install -q tensorboardX==2.6
!pip install -q tqdm==4.65.0
!pip install -q scikit-learn==1.3.2
!pip install -q 'spacy>=3.7.2,<3.8.0'
!pip install -q matplotlib==3.8.0
!pip install -q torch_geometric==2.4.0
!pip install -q 'pydantic>=2.7.0'

# Restart kernel to apply changes
import os
os.kill(os.getpid(), 9)

## 4. Import Libraries and Check Setup

In [ ]:
import torch
import numpy as np
import transformers
import os

print("=== Package Versions ===")
print(f"NumPy version: {np.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"\n=== GPU Information ===")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"\n=== Verification ===")
print(f"NumPy has dtypes: {hasattr(np, 'dtypes')} ✓")

## 5. Verify Data Files

In [ ]:
!ls -la data/14res/
!ls -la data/14lap/
!ls -la data/15res/
!ls -la data/16res/

## 6. Quick Test - Enhanced SemGCN + Boundary Refinement (1 epoch)
### Testing new boundary refinement module

In [ ]:
!python train.py \
    --seed 42 \
    --max_span_size 8 \
    --batch_size 16 \
    --epochs 1 \
    --dataset 14res \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 \
    --hidden_dim 384 \
    --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_boundary_refinement

## 7. Full Training - Enhanced SemGCN + Boundary Refinement (120 epochs)
### Expected: Triplet F1 77.5-77.7%, Entity F1 89.0-89.3%

In [ ]:
!python train.py \
    --seed 42 \
    --max_span_size 8 \
    --batch_size 16 \
    --epochs 120 \
    --dataset 14res \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 \
    --hidden_dim 384 \
    --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_boundary_refinement

## 8. Baseline Comparison - Enhanced SemGCN Only (No Boundary Refinement)
### For comparison: Expected Triplet F1 77.14%

In [ ]:
# Uncomment to run baseline (Enhanced SemGCN without boundary refinement)
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn

## 9. Test on Other Datasets

## 10. Test on Other Datasets

### 10a. Laptop 2014 Dataset

In [ ]:
# Uncomment to test on Laptop 2014 dataset
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14lap \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_contrastive \
#     --contrastive_weight 0.1

### 10b. Restaurant 2015 Dataset

In [ ]:
# Uncomment to test on Restaurant 2015 dataset
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 15res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_contrastive \
#     --contrastive_weight 0.1

## 11. View Training Logs

In [ ]:
!ls -lh log/14res/
!echo "\n=== Latest Training Log ==="
!tail -100 log/14res/train_*.log

## 12. Check Best Model Results

In [ ]:
import json
import glob

# Find latest results file
result_files = glob.glob('log/14res/*.json')
if result_files:
    latest_file = max(result_files, key=lambda x: os.path.getmtime(x))
    print(f"Latest results: {latest_file}\n")
    
    with open(latest_file, 'r') as f:
        results = json.load(f)
    
    print("=== Best Results ===")
    print(f"Entity F1: {results.get('entity_f1', 'N/A')}")
    print(f"Triplet F1: {results.get('triplet_f1', 'N/A')}")
    print(f"Best Epoch: {results.get('best_epoch', 'N/A')}")
else:
    print("No results found yet. Run training first.")

## 13. Performance Summary

### Expected Results:

| Configuration | Entity F1 | Triplet F1 | Improvement |
|--------------|-----------|------------|-------------|
| Baseline (Original) | 87.65% | 75.75% | --- |
| + Enhanced SemGCN | 88.68% | 77.14% | +1.39% |
| **+ Boundary Refinement** | **89.0-89.3%** | **77.5-77.7%** | **+1.75-1.95%** ✅ |

### Key Features:
- ✅ Enhanced Semantic GCN with relative position encoding
- ✅ Global context aggregation
- ✅ Multi-scale feature extraction
- ✅ **Boundary-aware attention for span refinement** (NEW)
- ✅ Separate start/end token attention

### Latest Improvement: Boundary Refinement
- 🎯 **Problem**: Imprecise span boundaries (e.g., "delicious pasta" vs "pasta")
- ✅ **Solution**: Boundary-aware attention focusing on start/end tokens
- 📈 **Expected Gain**: +0.4-0.6% Triplet F1

### Training Configuration:
```bash
python train.py --dataset 14res --epochs 120 \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 --hidden_dim 384 --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_boundary_refinement
```

### Note on Previous Experiments:
- ❌ Contrastive learning was tested but **decreased performance** (-1.04%)
- ✅ Enhanced SemGCN only: 77.14% Triplet F1
- ✅ **Current best**: Enhanced SemGCN + Boundary Refinement

### Next Steps to Reach 80% F1:
1. ✅ Boundary Refinement (+0.4-0.6%) - **CURRENT**
2. Cross-Attention Fusion (+0.5-0.7%)
3. Data Augmentation (+0.3-0.5%)
4. Ensemble Methods (+0.3-0.5%)

**Target**: 77.5% + 1.5-2.0% = **79.0-79.5%** (approaching 80%)